In [0]:
!pip install tf-nightly-gpu-2.0-preview

     |████████████████████████████████| 395.5MB 37kB/s 
     |████████████████████████████████| 3.8MB 47.1MB/s 
     |████████████████████████████████| 450kB 45.2MB/s 
     |████████████████████████████████| 81kB 11.6MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
ERROR: tb-nightly 2.1.0a20191206 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:


In [0]:
!pip uninstall -y kaggle

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6


In [0]:
!pip install kaggle==1.5.6

     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=4326a4314685b4eb928bb3202e3eba71eeb0ff4e2a9a90d16cc831ec4131db35
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
%env KAGGLE_USERNAME = ahmadsameh

env: KAGGLE_USERNAME=ahmadsameh


In [0]:
%env KAGGLE_KEY = 79c83ed6f294b92a11dcbff2ddc2e675

env: KAGGLE_KEY=79c83ed6f294b92a11dcbff2ddc2e675


In [0]:
!kaggle competitions download -c fcis-sc-deeplearning-competition

 98% 713M/724M [00:08<00:00, 87.0MB/s]
100% 724M/724M [00:08<00:00, 87.5MB/s]


In [0]:
!unzip "fcis-sc-deeplearning-competition.zip"

Archive:  fcis-sc-deeplearning-competition.zip
  inflating: submit.csv              
  inflating: test/test/003_1b.jpg    
  inflating: test/test/0310_piscines_8.jpg  
  inflating: test/test/051011Greenhouse.jpg  
  inflating: test/test/100_1238.jpg  
  inflating: test/test/1164876979_0.jpg  
  inflating: test/test/1208_hall_entrada_010.jpg  
  inflating: test/test/12_t1_H_S_Pool_Inside.jpg  
  inflating: test/test/13_2o_andar_panorama.jpg  
  inflating: test/test/1412_mb_file_0a8c5_gif.jpg  
  inflating: test/test/151815.jpg    
  inflating: test/test/1529445456_acec85121f.jpg  
  inflating: test/test/1_gymEmpty.jpg  
  inflating: test/test/2026_2006_Grimm_s_Gardens_Greenhouse.jpg  
  inflating: test/test/2333_2.jpg    
  inflating: test/test/250507130dh3.jpg  
  inflating: test/test/262507231_b0203fd5aa.jpg  
  inflating: test/test/28042007_001.jpg  
  inflating: test/test/281568964_bb524a3c9e_o.jpg  
  inflating: test/test/305015506_6df5016b5c.jpg  
  inflating: test/test/30_x_50_Po

In [0]:
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import cv2

train_path = "/content/train/train"
test_path = "/content/test/test"
Places = ["airport_inside", "bakery", "bedroom", "greenhouse", "gym", "kitchen",
          "operating_room", "poolinside", "restaurant","toystore"]
training_data = []
testing_data=[]

def create_training_data():
    for place in Places:
        path = os.path.join(train_path,place) 
        class_num = Places.index(place)  

        for imageName in tqdm(os.listdir(path)):
            try:
                img_array = cv2.imread(os.path.join(path,imageName) ) 
                new_array = cv2.resize(img_array, (224, 224))  
                training_data.append([new_array, class_num])  
            except Exception as e: 
                pass
 
def create_testing_data():
    for imageName in tqdm(os.listdir(test_path)):
            try:
                img_array = cv2.imread(os.path.join(test_path,imageName) ) 
                resizeImage = cv2.resize(img_array, (224, 224))  
                testing_data.append([resizeImage,imageName])  
            except Exception as e: 
                pass 

create_training_data()
create_testing_data()
print(len(training_data))
print(len(testing_data))

100%|██████████| 772/772 [00:05<00:00, 133.40it/s]

3137
771


In [0]:
from sklearn.model_selection import train_test_split
X_train = []
X_test = []
Y_train = [] 
img_names = []

for i in range(len(training_data)):
  X_train.append(training_data[i][0])
  Y_train.append(training_data[i][1])
for i in range(len(testing_data)):
  X_test.append(testing_data[i][0].astype(np.float64))
  img_names.append(testing_data[i][1])

X_train = np.asarray(X_train)
Y_train = np.asarray(Y_train)
X_test = np.asarray(X_test)
img_names = np.asarray(img_names)

print(len(Y_train))
X_train,X_validate,Y_train,Y_validate  = train_test_split(X_train,Y_train,test_size=0.3)
print(len(Y_train))

3137
2195


In [0]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import applications
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.models import Sequential

vgg19_model = applications.VGG19(weights='imagenet',input_shape=(224,224,3))
model = Sequential()
for layer in vgg19_model.layers[:-1]:
  model.add(layer)

for layer in model.layers:
  layer.trainable = False
model.add(Dense(10, activation='softmax'))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)      

In [0]:
from tensorflow.keras.optimizers import Adam , RMSprop,Adagrad
model.compile(Adam(lr=.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size = 32, epochs = 10, validation_data=(X_validate,Y_validate))

Train on 2195 samples, validate on 942 samples
Epoch 1/10
2195/2195 [==============================] - 22s 10ms/sample - loss: 0.7174 - accuracy: 0.7872 - val_loss: 0.4851 - val_accuracy: 0.8609
Epoch 2/10
2195/2195 [==============================] - 21s 10ms/sample - loss: 0.2412 - accuracy: 0.9226 - val_loss: 0.4815 - val_accuracy: 0.8652
Epoch 3/10
2195/2195 [==============================] - 22s 10ms/sample - loss: 0.1227 - accuracy: 0.9604 - val_loss: 0.4785 - val_accuracy: 0.8577
Epoch 4/10
2195/2195 [==============================] - 22s 10ms/sample - loss: 0.0707 - accuracy: 0.9809 - val_loss: 0.4400 - val_accuracy: 0.8662
Epoch 5/10
2195/2195 [==============================] - 22s 10ms/sample - loss: 0.0406 - accuracy: 0.9923 - val_loss: 0.4287 - val_accuracy: 0.8737
Epoch 6/10
2195/2195 [==============================] - 22s 10ms/sample - loss: 0.0237 - accuracy: 0.9986 - val_loss: 0.4451 - val_accuracy: 0.8737
Epoch 7/10
2195/2195 [==============================] - 22s 10ms/

In [0]:
predictions = model.predict(X_test)
print(predictions)
labels = []
for i in predictions:
  labels.append(np.argmax(i)+1)

labels = np.asarray(labels)


[[9.8295814e-01 7.1674114e-04 2.4080866e-03 ... 4.8762196e-04
  6.4770756e-03 3.8479902e-05]
 [3.3104207e-06 5.0487057e-07 1.1389809e-10 ... 2.3089103e-07
  1.0365122e-06 9.9998724e-01]
 [1.0058418e-04 2.0415961e-05 1.7280051e-02 ... 9.7944301e-01
  4.4914859e-04 6.3784756e-07]
 ...
 [8.4474407e-02 3.9878311e-03 4.8574931e-09 ... 1.1953912e-03
  8.9159310e-01 8.5753193e-03]
 [2.1799380e-09 1.3389879e-08 8.9690268e-08 ... 4.5397965e-12
  9.9999845e-01 1.2182430e-10]
 [3.8781130e-07 2.7536767e-10 5.2906685e-10 ... 9.9999893e-01
  7.3191373e-08 1.4156680e-08]]


In [0]:
import pandas as pd

missing = '1412_mb_file_0a8c5_gif.jpg'
img_names=np.append(img_names,missing)
labels=np.append(labels,4)
data = {'Id':img_names , 'Label':labels} 
df = pd.DataFrame(data)
export_csv = df.to_csv('/content/submit.csv',index = None)